Modelos basados en N-Gramas
=========================

Introducción
----------------

### Para ejecutar este notebook

Para ejecutar este notebook, instale las siguientes librerias:

In [11]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv \
    --quiet --no-clobber --directory-prefix ./Datasets/mascorpus/

!wget https://raw.githubusercontent.com/santiagxf/M72109/master/m72109/nlp/normalization.py \
    --quiet --no-clobber --directory-prefix ./m72109/nlp/
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/m72109/nlp/transformation.py \
    --quiet --no-clobber --directory-prefix ./m72109/nlp/
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/docs/nlp/neural/ngram-cnn.txt \
    --quiet --no-clobber
!pip install -r ngram-cnn.txt --quiet

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'ngram-cnn.txt'


Importamos algunas librerias necesarias

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [2]:
tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

## Preprocesamiento de texto

Al igual que con Topic Modeling, nuestro primer paso es preprocesar el texto. Para focalizarnos en Word2Vec en este modulo, les preparé un modulo TweetTextNormalizer que hará todo el preprocesamiento por nosotros. Pueden explorar los parametros que recibe el constructor de esta clase para ver que opciones podemos configurar como Stemmer, Lemmatization, etc.

En lo particular, estamos creando un TweetTextNormalizer que:
 - Aplicará un tokenizer especifico para Twitter
 - Eliminará stop words
 - Aplicará lemmatization
 - Eliminará URLs
 - Eliminará acentos
 - Eliminará las mayusculas
 
Adicionalmente, el parametro text_to_sequence=True indica que la salida de este proceso no serán oraciones sino que tokens.

In [3]:
from m72109.nlp.normalization import TweetTextNormalizer

In [4]:
normalizer = TweetTextNormalizer(preserve_case=False, return_tokens=True)

## Vectorización de las palabras

En las actividades anteriores utilizamos siempre un TF-IDF vectorizer para generar los vectores. En esta oportunidad utilizaremos Word2Vec utilizando un modelo pre-entrenado para el idioma español. Adicionalmente, vemos que este vectorizer tiene el parametro sequence_to_idx en Verdadero. Esto significa que no queremos que como salida obtengamos los vectores de Word2Vec, sino que queremos "el indice" que se corresponde a la palabra en una matriz de indice-palabra/vectores. 

In [5]:
from m72109.nlp.transformation import Word2VecVectorizer

In [6]:
w2v = Word2VecVectorizer(model_path='Models/Word2Vec/model-es.bin', sequence_to_idx=True)
embedding_weights = w2v.get_weights()

### Ajustando la longitud de las secuencias

Los modelos basados en secuencias pueden adaptarse a cualquier longitud de secuencia, sin embargo, los parametros de nuestras redes neuronales deberan ser fijos. Para esto definiermos una longitud máxima de la secuencia que vamos analizar. Para esto podemos utilizar un valor especifico o utilizar el valor máximo de tokens que hay en nuestro corpus.

La siguiente clase PadSequenceTransformer es un modulo que les preparé para simplificar este procesamiento. El mismo se encarga de ajustar cualquier secuencia para que tenga exactamente max_seq_len. Cuando la lingitud es mejor, se completan con ceros.

In [7]:
from m72109.nlp.transformation import PadSequenceTransformer

In [8]:
max_seq_len = 100

In [9]:
seq2seq = PadSequenceTransformer(max_len=max_seq_len)

El método get_weights() construye la matríz de indice-palabra/vector que luego será utilizado para encontrar los vectores correspondientes de cada palabra. Esta matriz tiene dimensiones m x n, donde m es la cantidad de palabras del vocabulario y n la dimensión de los vectores de word2vec. En este caso trabajamos con vectores de dimensionalidad 100.

In [10]:
vocab_size = w2v.vocab_size
embedding_dim = w2v.emdedding_size

batch_size = 50
feature_maps = 100
n_gram = 3

In [22]:
def build_model(max_len, vocab_size, embedding_dim, embedding_weights):
    model = keras.Sequential() #(batch_size, max_len)
    model.add(keras.layers.Embedding(vocab_size, embedding_dim, input_shape=(max_len,)), trainable=False, mask_zero=True) #(batch_size, max_len, embedding)
    model.add(keras.layers.Conv1D(filters = feature_maps, kernel_size = n_gram, activation = keras.activations.relu))
    model.add(keras.layers.MaxPooling1D(pool_size = batch_size - n_gram + 1, strides = None, padding = 'valid'))
    model.add(keras.layers.GlobalAveragePooling1D())
    model.add(keras.layers.Dense(7, activation=keras.activations.softmax))

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [23]:
estimator = keras.wrappers.scikit_learn.KerasClassifier(
    build_fn=build_model, 
    epochs=50,
    max_len=max_seq_len,
    vocab_size=vocab_size,
    embedding_dim=w2v.emdedding_size,
    embedding_weights=embedding_weights)

## Creando nuestro pipeline

Construimos nuestro pipeline como siempre combinando los diferentes pasos:

In [24]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(steps=[('normalizer', normalizer), 
                           ('vectorizer', w2v),
                           ('padder', seq2seq),
                           ('estimator', estimator)])

Entrenamos nuestro modelo

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets['TEXTO'], tweets['SECTOR'], 
                                                    test_size=0.33, 
                                                    stratify=tweets['SECTOR'])

In [ ]:
model = pipeline.fit(X=X_train, y=y_train)

## Evalución de los resultados

Probamos su performance utilizando el test set

In [ ]:
predictions = model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))